In [3]:
import os
os.getcwd()

'c:\\Users\\j_m289\\Pictures\\PHD\\7. Data Analysis\\HMR\\Code'

In [5]:
#Load packages
import os
import pandas as pd #pandas for data frame management and descriptives (in addition to numpy)
import seaborn as sns #seaborn for visualisation (MATPLOTLIB also)
import scipy as sci 
from scipy import stats
from scipy.stats import iqr 
from statannotations.Annotator import Annotator
import numpy as np
import matplotlib.pyplot as plt
import pingouin as png 
import xlsxwriter
OUTDATED_IGNORE=1

os.chdir('c:\\Users\\j_m289\\Pictures\\PHD\\7. Data Analysis\\HMR\\Clean')  # Directory set to 'clean' folder

#check and set working directory
cwd = os.getcwd()  # Get the current working directory (cwd)
files = os.listdir(cwd)  # Get all the files in that directory
print("Files in %r: %s" % (cwd, files))

#try to use relative path for files (tied to directory and nesting within folders)
path = "/Users/j_m289\Pictures\PHD\15. Data Analysis\HMR\Clean"
start = "/Users/j_m289"
relative_path = os.path.relpath(path, start)

# Set plot themes
sns.set_theme(style="whitegrid")

Files in 'c:\\Users\\j_m289\\Pictures\\PHD\\7. Data Analysis\\HMR\\Clean': ['.Rhistory', '09F2F820', 'AnorexiaDataset_Final.csv', 'BingeEatingDataset_Final.csv', 'BulimiaDataset_Final.csv', 'HealthyDataset_Final.csv', 'OsfedDataset_Final.csv']


# Descriptive Statistics Across Groups
- Functions defined for descriptive stats of continous and categorical variables.
- Lists defined for continous and categorical variables.

In [6]:
AN = pd.read_csv('AnorexiaDataset_Final.csv', index_col=[0]) # Anorexia Nervosa
BN = pd.read_csv('BulimiaDataset_Final.csv', index_col=[0]) # Bulimia Nervosa
BED = pd.read_csv('BingeEatingDataset_Final.csv', index_col=[0]) # Binge Eating Disorder
OSFED = pd.read_csv('OsfedDataset_Final.csv', index_col=[0]) # OSFED
healthy = pd.read_csv('HealthyDataset_Final.csv', index_col=[0]) # Healthy

# Define columns from dataset needed for analysis by variable type
#Continuous 
correlations  =['Age', 'BMI','EmReg_CR','EmReg_ES','CFI_Alt', 'CFI_Ctrl','YPSQ_EFSA','YPSQ_S','YPSQ_EC','YPSQ_O','YPSQ_EOS','YPSQ_SCRE','YPSQ_DS','YPSQ_SB','YPSQ_HSC','EDE_Q_R','EDE_Q_W',\
                'EDE_Q_S','EDE_Q_E', 'EDE_Q_G']

demo_cont = ['Age','Height','Weight','BMI']

scales_cont = ['Appearance_Satisfact','YPSQ_EFSA','YPSQ_S','YPSQ_EC','YPSQ_O','YPSQ_EOS','YPSQ_SCRE','YPSQ_DS','YPSQ_SB','YPSQ_HSC','EmReg_CR','EmReg_ES','EDE_Q_R','EDE_Q_W',\
    'EDE_Q_S','EDE_Q_E', 'EDE_Q_G','CFI_Alt', 'CFI_Ctrl']

model = ['EDE_Q_G','YPSQ_EFSA','YPSQ_S','YPSQ_EC','YPSQ_O','YPSQ_EOS','YPSQ_SCRE','YPSQ_DS','YPSQ_SB','YPSQ_HSC','EmReg_CR','EmReg_ES','CFI_Alt', 'CFI_Ctrl']

#Categorical
demo_cat = ['Sex','Education','Nationality','MH_Diag','EatD','MDD','BiP','SAD','GAD','PaD','PTSD','SUD','OCD','BDD','BPD','ADHD','Other','MH_ND','MHBurden','ED_Diag',\
    'ED_subtype','past_ED_Diag','pastAN','pastBN','pastBED','partOSFED','ED_treatment']

#Define categorical variable master dictionary
nested_dict = { 'Sex': {'Male': 1, 'Female': 2, 'ND': 3},
                'Education': {'DNC Gr10': 1, 'Grd10': 2, 'Grd12':3, 'VET': 4, 'Bchl': 5, 'Mast':6,'Phd':7},
                'Nationality': {'Australian': 1, 'New Zealand': 2, 'North American':3, 'South American': 4, 'British': 5, 'Chinese':6,'Other':7, 'Canadian':8},
                'MH_Diag': {'Yes': 1, 'No': 2, 'ND':3},
                'EatD': {'No': 0, 'Yes': 1},
                'MDD': {'No': 0, 'Yes': 1},
                'BiP': {'No': 0, 'Yes': 1},
                'SAD': {'No': 0, 'Yes': 1},
                'GAD': {'No': 0, 'Yes': 1},
                'PaD': {'No': 0, 'Yes': 1},
                'PTSD': {'No': 0, 'Yes': 1},
                'SUD': {'No': 0, 'Yes': 1},
                'OCD': {'No': 0, 'Yes': 1},
                'BDD': {'No': 0, 'Yes': 1},
                'BPD': {'No': 0, 'Yes': 1},
                'ADHD': {'No': 0, 'Yes': 1},
                'Other': {'No': 0, 'Yes': 1},
                'MH_ND': {'No': 0, 'Yes': 1},
                'MHBurden': {'None': 0, 'One': 1, 'Two':2, 'Three': 3, 'Four': 4, 'Five':5, 'Six': 6, 'Seven': 7, 'Eight':8, 'Nine': 9, 'Ten': 10, 'Eleven':11, 'Twelve': 12, 'Thirteen':13},
                'ED_Diag': {'Yes': 1, 'No': 2, 'ND':3},
                'ED_subtype': {'AN': 1, 'BN':2, 'BED': 3, 'OSFED': 4, 'ND':5},
                'past_ED_Diag': {'Yes': 1, 'No': 2, 'ND':3},
                'pastAN': {'No': 0, 'Yes': 1},
                'pastBN': {'No': 0, 'Yes': 1},
                'pastBED': {'No': 0, 'Yes': 1},
                'partOSFED': {'No': 0, 'Yes': 1},
                'ED_treatment': {'Previously': 1, 'Current': 2, 'Never':3, 'ND': 4}
                }

# Functions
def descriptive_stats(data, name):
    # create a dataframe to store the results
    results = pd.DataFrame(columns=['Name','Mean (SD)','Median (IQR)','Mode','Variance', 'Min', 'Max','Range'])
    # create a dictionary to store the statistics
    stats = {'Name': name, 'Mean (SD)': str(data.mean().round(2)) + ' (' + str(data.std().round(2)) + ')', 'Median (IQR)': str(data.median().round(2)) + ' (' + str(iqr(data).round(2)) + ')', 'Mode': data.mode().values[0], 'Variance': data.var().round(2),\
    'Min': data.min(), 'Max': data.max(), 'Range': data.max() - data.min()}
    # append the dictionary to the results dataframe
    return results.append(stats, ignore_index=True) 

def descriptive_stats_freq(data, name, dict):
    results = pd.DataFrame(columns=['Keys','Count (%)'])
    for key, val in dict.items():
        count = len(data[data == val])
        percent = round(count / len(data)*100,2)
        results = results.append({'Keys': key, 'Count (%)': str(count) + ' (' +  str(percent) + ')',}, ignore_index=True)
    display(name, results)

In [7]:
# The following creates dataset with all groups and assigns a column with sub-type
# Create columns to assign string and value (used by ANOVA as between-subjects factor)
AN['subtype'] = 'AN'
BN['subtype'] = 'BN'
BED['subtype'] = 'BED'
OSFED['subtype'] = 'OSFED'
healthy['subtype'] = 'Healthy'

AN['subtype#'] = 1
BN['subtype#'] = 2
BED['subtype#'] = 3
OSFED['subtype#'] = 4
healthy['subtype#'] = 5

# Concatenate datasets and set keys
full_dataset = pd.concat([AN, BN, BED, OSFED, healthy], keys=['AN', 'BN', 'BED', 'OSFED', 'healthy'])

# Change directory to save outputs
os.chdir('c:\\Users\\j_m289\\Pictures\\PHD\\7. Data Analysis')

In [ ]:
full_dataset.to_csv('FullDataset.csv', index=True) #save dataset

# Demographics Categories

In [ ]:
for x in demo_cat:
      descriptive_stats_freq(AN[x], x, nested_dict[x])

In [ ]:
for x in demo_cat:
    descriptive_stats_freq(BN[x], x, nested_dict[x])

In [ ]:
for x in demo_cat:
    descriptive_stats_freq(BED[x], x, nested_dict[x])

In [ ]:
for x in demo_cat:
    descriptive_stats_freq(OSFED[x], x, nested_dict[x])

In [ ]:
for x in demo_cat:
    descriptive_stats_freq(healthy[x], x, nested_dict[x])

# Demographics (Continous)

In [ ]:
# Anorexia Nervosa
AN_demo_cont_list = [] # create a list to store the results of each demographic continous variables
for x in demo_cont:
    result = descriptive_stats(AN[x], x)
    AN_demo_cont_list.append(result)

# concatenate the list of dataframes into a single dataframe
AN_demo_cont_output = pd.concat(AN_demo_cont_list)
display(AN_demo_cont_output)

#Bulimia Nervosa
BN_demo_cont_list = []
for x in demo_cont:
    result = descriptive_stats(BN[x], x)
    BN_demo_cont_list.append(result)

# concatenate the list of dataframes into a single dataframe
BN_demo_cont_output = pd.concat(BN_demo_cont_list)
display(BN_demo_cont_output)

#Binge Eating Disorder
BED_demo_cont_list = []
for x in demo_cont:
    result = descriptive_stats(BED[x], x)
    BED_demo_cont_list.append(result)

# concatenate the list of dataframes into a single dataframe
BED_demo_cont_output = pd.concat(BED_demo_cont_list)
display(BED_demo_cont_output)

#OSFED
OSFED_demo_cont_list = []
for x in demo_cont:
    result = descriptive_stats(OSFED[x], x)
    OSFED_demo_cont_list.append(result)

# concatenate the list of dataframes into a single dataframe
OSFED_demo_cont_output = pd.concat(OSFED_demo_cont_list)
display(OSFED_demo_cont_output)

#Healthy
healthy_demo_cont_list = []
for x in demo_cont:
    result = descriptive_stats(healthy[x], x)
    healthy_demo_cont_list.append(result)

# concatenate the list of dataframes into a single dataframe
healthy_demo_cont_output = pd.concat(healthy_demo_cont_list)
display(healthy_demo_cont_output)

In [ ]:
writer = pd.ExcelWriter('Demo_cont_inferential.xlsx', engine='xlsxwriter')
significant_comp = []

# One-way comparisons using Welch method (unequal N and heterogeneity), box plots and post-hoc comparisons for demo cont variables
for i in demo_cont:
    # One way ANOVA 
    welch = png.welch_anova(data = full_dataset, dv = str(i), between='subtype')
    png.print_table(welch, floatfmt='.3f')

    # Post Hoc (if significant)
    posthoc = png.pairwise_gameshowell(data = full_dataset, dv = str(i), between='subtype', effsize='eta-square')
    png.print_table(posthoc, floatfmt='.3f')
    temp_name = i + ' Post Hoc'
    temp_figname = i + ' Figure'

    # Check for significant p-value and append to list
    for a, b, p in zip(posthoc.A, posthoc.B, posthoc.pval):
        if p < 0.05:
            comparison_name = f"({a} , {b})"
            significant_comp.append(comparison_name)
    
    # post_hoc_temp = ', '.join(str(significant_comp))
    print(significant_comp)

    # Box and whisker plot by group
    fig, ax = plt.subplots(1, 1)
    ax.boxplot([AN[i], BED[i], BN[i], OSFED[i], healthy[i]])
    ax.set_xticklabels(['AN', 'BED', 'BN', 'OSFED', 'healthy']) 
    ax.set_ylabel(i) 
    plt.show()
    
    x = full_dataset['subtype']
    order = ['AN', 'BN', 'BED', 'OSFED', 'Healthy']
    boxplot = sns.boxplot(data=full_dataset, x=x, y=i, hue=None, order=None, hue_order=None, orient=None)
    figsave = i + '_boxplot.png'
    plt.savefig(figsave)
    plt.show()
    
    # annotator = Annotator(ax, significant_comp, data=full_dataset, x=x, y=i, order=order)
    # annotator.configure(test="Mann-Whitney", text_format="simple")
    # annotator.apply_and_annotate()
    
    # save results to a separate sheet in the Excel file
    welch.to_excel(writer, sheet_name=i, index=True)
    posthoc.to_excel(writer, sheet_name=temp_name, index=True)
    worksheet = writer.sheets[temp_name]
    worksheet.insert_image('C15',figsave)
    # boxplot.to_excel(writer, sheet_name=temp_figname)

# save the Excel file
writer.save()
writer.close()

# Scale Variables

In [ ]:
# Anorexia Nervosa
AN_scales_cont_list = [] # create a list to store the results of each continous predictor and dependent variable
for x in scales_cont:
    result = descriptive_stats(AN[x], x)
    AN_scales_cont_list.append(result)

# concatenate the list of dataframes into a single dataframe
AN_scales_cont_output = pd.concat(AN_scales_cont_list)
display(AN_scales_cont_output)

#Bulimia Nervosa
BN_scales_cont_list = []
for x in scales_cont:
    result = descriptive_stats(BN[x], x)
    BN_scales_cont_list.append(result)

# concatenate the list of dataframes into a single dataframe
BN_scales_cont_output = pd.concat(BN_scales_cont_list)
display(BN_scales_cont_output)

#Binge Eating Disorder
BED_scales_cont_list = []
for x in scales_cont:
    result = descriptive_stats(BED[x], x)
    BED_scales_cont_list.append(result)

# concatenate the list of dataframes into a single dataframe
BED_scales_cont_output = pd.concat(BED_scales_cont_list)
display(BED_scales_cont_output)

#OSFED
OSFED_scales_cont_list = []
for x in scales_cont:
    result = descriptive_stats(OSFED[x], x)
    OSFED_scales_cont_list.append(result)

# concatenate the list of dataframes into a single dataframe
OSFED_scales_cont_output = pd.concat(OSFED_scales_cont_list)
display(OSFED_scales_cont_output)

#Healthy
healthy_scales_cont_list = []
for x in scales_cont:
    result = descriptive_stats(healthy[x], x)
    healthy_scales_cont_list.append(result)

# concatenate the list of dataframes into a single dataframe
healthy_scales_cont_output = pd.concat(healthy_scales_cont_list)
display(healthy_scales_cont_output)

In [ ]:
writer = pd.ExcelWriter('Scales_cont_inferential.xlsx', engine='xlsxwriter')
significant_comp = []
# One-way comparisons using Welch method (unequal N and heterogeneity), box plots and post-hoc comparisons for demo cont variables
for i in scales_cont:
    # One way ANOVA 
    welch = png.welch_anova(data = full_dataset, dv = str(i), between='subtype')
    png.print_table(welch, floatfmt='.3f')

    # Post Hoc (if significant)
    # print(welch.pcorr)
    # if welch.pcorr < 0.05:
    posthoc = png.pairwise_gameshowell(data = full_dataset, dv = str(i), between='subtype', effsize='eta-square')
    png.print_table(posthoc, floatfmt='.3f')
    temp_name = i + ' Post Hoc'
    temp_figname = i + ' Figure'

    # Check for significant p-value and append to list
    for a, b, p in zip(posthoc.A, posthoc.B, posthoc.pval):
        if p < 0.05:
            comparison_name = f"({a} , {b})"
            significant_comp.append(comparison_name)
    
    # post_hoc_temp = ', '.join(str(significant_comp))
    print(significant_comp)

    # Box and whisker plot by group
    fig, ax = plt.subplots(1, 1)
    ax.boxplot([AN[i], BED[i], BN[i], OSFED[i], healthy[i]])
    ax.set_xticklabels(['AN', 'BED', 'BN', 'OSFED', 'healthy']) 
    ax.set_ylabel(i) 
    plt.show()
    
    x = full_dataset['subtype']
    order = ['AN', 'BN', 'BED', 'OSFED', 'Healthy']
    boxplot = sns.boxplot(data=full_dataset, x=x, y=i, hue=None, order=None, hue_order=None, orient=None)
    figsave = i + '_boxplot.png'
    plt.savefig(figsave)
    plt.show()
    
    # annotator = Annotator(ax, significant_comp, data=full_dataset, x=x, y=i, order=order)
    # annotator.configure(test="Mann-Whitney", text_format="simple")
    # annotator.apply_and_annotate()
    
    # save results to a separate sheet in the Excel file
    welch.to_excel(writer, sheet_name=i, index=True)
    posthoc.to_excel(writer, sheet_name=temp_name, index=True)
    worksheet = writer.sheets[temp_name]
    worksheet.insert_image('C15',figsave)
    # boxplot.to_excel(writer, sheet_name=temp_figname)

# save the Excel file
writer.save()
writer.close()

# Correlations

In [36]:
# 24/01/2024 - Updated code to use pairwisecorr instead of rcorr (in line with reviewer comments)

writer = pd.ExcelWriter('Correlations.xlsx', engine='xlsxwriter')
list = full_dataset['subtype'].unique()
for v in list:
    data = full_dataset.loc[full_dataset['subtype'] == v]
    # Correlation matrix for model variables (excluding sex)
    output = png.pairwise_corr(data, columns=correlations, method='pearson', padjust='bonf').round(4)
    output = output.drop(axis=1, columns=['CI95%','alternative',"method", 'n', 'p-adjust', 'BF10', 'power'])
    # Export dataframe to excel in different sheets
    output.to_excel(writer, sheet_name='Model Predictors' + v , index=True)
    
writer.save()
writer.close()

c:\Users\j_m289\AppData\Local\Programs\Python\Python39\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [ ]:
# The following is draft code for semi-partial correlation calculation
scales_cont_copy = scales_cont
Temp_scales = scales_cont_copy.remove('EDE_Q_G')
for subtype in list:
    data = full_dataset[full_dataset['subtype'] == subtype]
    for i in scales_cont:
        covar = [x for x in scales_cont if x != i]
        print(png.partial_corr(data= data, x=i, y='EDE_Q_G', x_covar=covar).round(3))